In [45]:
import pandas as pd
import numpy as np
import json
import geojson

In [32]:
#import climate change data
df = pd.read_csv('./raw/climate_change_orig.csv')

#delete not used columns
df.drop(['Domain Code', 'Area Code', 'Domain', 'Element Code', 'Element', 'Months Code', 'Months', 'Year Code', 'Unit', 'Flag', 'Flag Description'], axis=1, inplace=True)

#rename columns
df.columns = ['country_eng', 'year', 'value']

df.head()

,country_eng,year,value
0,Afghanistan,1961,-0.080
1,Afghanistan,1962,-0.120
2,Afghanistan,1963,0.882
3,Afghanistan,1964,-0.727
4,Afghanistan,1965,-0.200


In [33]:
#import country codes for mapping
df_country_code = pd.read_csv('./raw/country_codes.CSV', index_col=5)

#delete not used columns
df_country_code.drop(['LANG', 'LANG_NAME', 'COUNTRY_ALPHA3_CODE', 'COUNTRY_NUMERIC_CODE'], axis=1, inplace=True)

#create dict: (<countryname>: <countycode>)
country_dict = df_country_code.to_dict('dict')


In [34]:
def get_country_code(country_name, country_dict):
    """returns the country code of specific country
    
        Parameters:
        country_name: Name of the country in english (String)
        country_dict: Dictionary with country - country code mapping (Dictionary)

        Returns: country code if found, otherwise empty string (String)

       """    
    for key, value in country_dict.get('COUNTRY_ALPHA2_CODE').items():
        if key in country_name:
            return str(value).lower()
    return ""

In [35]:
def get_country_translation(country_code, country_dict):
    """returns the translated country name
    
        Parameters:
        country_code: Alpha2 Country Code (String)
        country_dict: Dictionary with country code - translated country name mapping (Dictionary)

        Returns: translated country name if found, otherwise empty String (String)

       """    

    for key, value in country_dict.items():
        if key in country_code:
            return str(value)
    return ""

In [36]:
#mapping (country -> country code) & add to dataframe
df['country_code'] = df['country_eng'].apply(lambda x: get_country_code(x, country_dict))

In [37]:
#import french translated country name
df_de = pd.read_csv('./raw/country_translation_de.csv', usecols=['name', 'alpha2'])
df_fr = pd.read_csv('./raw/country_translation_fr.csv', usecols=['name', 'alpha2'])
df_it = pd.read_csv('./raw/country_translation_it.csv', usecols=['name', 'alpha2'])

#create dict: (<countycode>:<translated countryname>)
country_dict_de = df_de.set_index('alpha2')['name'].to_dict()
country_dict_fr = df_fr.set_index('alpha2')['name'].to_dict()
country_dict_it = df_it.set_index('alpha2')['name'].to_dict()

In [38]:
#mapping (country code -> translated country name) & add to dataframe
df['country_de'] = df['country_code'].apply(lambda x: get_country_translation(x, country_dict_de))
df['country_fr'] = df['country_code'].apply(lambda x: get_country_translation(x, country_dict_fr))
df['country_it'] = df['country_code'].apply(lambda x: get_country_translation(x, country_dict_it))

df.head()

,country_eng,year,value,country_code,country_de,country_fr,country_it
0,Afghanistan,1961,-0.080,af,Afghanistan,Afghanistan,Afghanistan
1,Afghanistan,1962,-0.120,af,Afghanistan,Afghanistan,Afghanistan
2,Afghanistan,1963,0.882,af,Afghanistan,Afghanistan,Afghanistan
3,Afghanistan,1964,-0.727,af,Afghanistan,Afghanistan,Afghanistan
4,Afghanistan,1965,-0.200,af,Afghanistan,Afghanistan,Afghanistan


In [39]:
#save as a new csv file
df.to_csv("./climate_change_cleaned.csv")

In [48]:
#add geojson
with open('./raw/ne_110m_admin_0_countries.geojson', 'r') as f:
    geodata = geojson.load(f)

In [49]:
#NEW
def get_temperature(country_code, df):
    """returns the temperature of specific country with country code
    
        Parameters:
        country_code: ISO alpha 2 Country Code (String)
        df: dataframe with country code and temperature values (Dataframe)

        Returns: temperature value if found, otherwise 999 (Str)

       """    
    for index, row in df.iterrows():
        if row['country_code'].lower() == country_code.lower() and row['year'] == 2019:
            return str(row['value'])
    return 999

In [50]:
#add temp value to geojson properties
for feat in geodata['features']:
    feat['properties']['TEMP'] = get_temperature(feat['properties']['ISO_A2'], df)

In [51]:
#Write result to a new file
with open('./geojson_temperature_str.geojson', 'w') as f:
    geojson.dump(geodata, f)